In [6]:
import sklearn
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import optuna
import mlflow
from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima
from keras.models import load_model
import joblib
from weatherapi import WeatherPoint
import requests
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pickle
from pmdarima.arima import auto_arima
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from keras.models import load_model
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import xgboost as xgb

ModuleNotFoundError: No module named 'statsmodels'

http://localhost:8070

In [7]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch4",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch4",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 12
cpu = 12*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         # .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.jars.packages", ','.join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         # .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.21.172") # from GCP VM instances or other server ip
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default")
         # .config("spark.spark.clickhouse.write.compression.codec", "lz4")
         # .config("spark.clickhouse.read.compression.codec", "lz4")
         # .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         .config("spark.clickhouse.write.batchSize", "10000000")
         # .config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         # .config("spark.clickhouse.options.socket_timeout", "600000000")
         # .config("spark.clickhouse.options.connection_timeout", "600000000")
         # .config("spark.clickhouse.options.query_timeout", "600000000")
         .config("spark.executor.memory", f"{ram}g")
         .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         #.config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

24/05/12 20:42:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[]

In [3]:
df.show()

NameError: name 'df' is not defined

In [4]:
spark.sql('show databases').show()

+------------------+
|         namespace|
+------------------+
|INFORMATION_SCHEMA|
|           default|
|    first_database|
|information_schema|
|            system|
+------------------+



In [3]:
#spark.sql('CREATE DATABASE first_database').show()

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `first_database` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [4]:
df = spark.read.csv('/projects/app/stations.csv',header=True,inferSchema=True)

In [2]:
df = spark.sql('''SELECT * FROM first_database.WeatherData''')

In [5]:

(
    df
    .write
    .mode("append")
    .format("jdbc")
    .option("url", "jdbc:clickhouse://34.118.0.142:9000/")
    .option("driver", "com.github.housepower.jdbc.ClickHouseDriver")
    .option("user", "default")
    .option("password", "1234")
    .option("dbtable", "first_database.WeatherDataSpark")
    .option("batchsize", "10000000")
    .option("numPartitions", cpu)
    .option("truncate", "true")
    .option("isolationLevel", "NONE")
    .save()
)


Py4JJavaError: An error occurred while calling o94.save.
: com.github.housepower.exception.InvalidValueException: Connection is not support
	at com.github.housepower.jdbc.ClickhouseJdbcUrlParser.parseJdbcUrl(ClickhouseJdbcUrlParser.java:56)
	at com.github.housepower.settings.ClickHouseConfig$Builder.withJdbcUrl(ClickHouseConfig.java:355)
	at com.github.housepower.jdbc.ClickHouseDriver.connect(ClickHouseDriver.java:48)
	at com.github.housepower.jdbc.ClickHouseDriver.connect(ClickHouseDriver.java:26)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.exe

In [3]:
df.createOrReplaceTempView('df_sql')
spark.sql("""
INSERT INTO first_database.WeatherDataSpark
SELECT * FROM df_sql
""")

24/04/28 08:55:57 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 1]
java.lang.NoSuchMethodError: 'org.apache.arrow.vector.types.pojo.Schema org.apache.spark.sql.util.ArrowUtils$.toArrowSchema(org.apache.spark.sql.types.StructType, java.lang.String)'
	at org.apache.spark.sql.clickhouse.SparkUtils$.toArrowSchema(SparkUtils.scala:28)
	at xenon.clickhouse.write.format.ClickHouseArrowStreamWriter.<init>(ClickHouseArrowStreamWriter.scala:31)
	at xenon.clickhouse.write.ClickHouseBatchWrite.createWriter(ClickHouseWrite.scala:73)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(WriteToDataSourceV2Exec.scala:491)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.$anonfun$writeWithV2$2(WriteToDataSourceV2Exec.scala:388)
	at org.apa

Py4JJavaError: An error occurred while calling o57.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (ae0a349aa8da executor driver): java.lang.NoSuchMethodError: 'org.apache.arrow.vector.types.pojo.Schema org.apache.spark.sql.util.ArrowUtils$.toArrowSchema(org.apache.spark.sql.types.StructType, java.lang.String)'
	at org.apache.spark.sql.clickhouse.SparkUtils$.toArrowSchema(SparkUtils.scala:28)
	at xenon.clickhouse.write.format.ClickHouseArrowStreamWriter.<init>(ClickHouseArrowStreamWriter.scala:31)
	at xenon.clickhouse.write.ClickHouseBatchWrite.createWriter(ClickHouseWrite.scala:73)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(Wri

In [ ]:
spark.sql("""
SELECT * FROM default.input_test_table
""").show()

In [7]:
#df.createOrReplaceTempView('df_sql')
df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""").show()

+--------------------+----+-----+-----+----+----+--------------------+---+----+---+--------------------+-------------------+----------+--------------------+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+---+----+----------------+----+--------------------+---+---+---+
|Local time in Moscow|   T|   Po|    P|  Pa|   U|                  DD| Ff|ff10|ff3|                   N|                 WW|        W1|                  W2|  Tn|  Tx|                  Cl|                  Nh|                   H|                  Cm|                  Ch| VV|  Td|             RRR|  tR|                   E| Tg| E'|sss|
+--------------------+----+-----+-----+----+----+--------------------+---+----+---+--------------------+-------------------+----------+--------------------+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+---+----+----------------+----+--------------------+---+-

In [13]:
df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""")

In [16]:
df.show()

+--------------------+----+-----+-----+----+----+--------------------+---+----+---+--------------------+-------------------+----------+--------------------+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+---+----+----------------+----+--------------------+---+---+---+
|Local time in Moscow|   T|   Po|    P|  Pa|   U|                  DD| Ff|ff10|ff3|                   N|                 WW|        W1|                  W2|  Tn|  Tx|                  Cl|                  Nh|                   H|                  Cm|                  Ch| VV|  Td|             RRR|  tR|                   E| Tg| E'|sss|
+--------------------+----+-----+-----+----+----+--------------------+---+----+---+--------------------+-------------------+----------+--------------------+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+---+----+----------------+----+--------------------+---+-

In [12]:
from pyspark.sql import SparkSession

# Создание сессии Spark
# spark = SparkSession.builder \
#     .appName("Temperature Forecasting") \
#     .getOrCreate()

# # Загрузка данных из базы данных в Spark DataFrame
# jdbc_url = "jdbc:postgresql://your_database_host:your_database_port/your_database_name"
# table_name = "your_table_name"
# user = "your_username"
# password = "your_password"
# 
# df = spark.read.format("jdbc") \
#     .option("url", jdbc_url) \
#     .option("dbtable", table_name) \
#     .option("user", user) \
#     .option("password", password) \
#     .load()

# Предобработка данных
from pyspark.sql.functions import year, month, dayofmonth, hour

df = df.withColumn("Local time in Moscow", to_timestamp("Local time in Moscow", "dd.MM.yyyy HH:mm")) \
    .filter(df["Local time in Moscow"] < "2023-01-01")

df = df.withColumn("year", year("Local time in Moscow")) \
    .withColumn("month", month("Local time in Moscow")) \
    .withColumn("day", dayofmonth("Local time in Moscow")) \
    .withColumn("hour", hour("Local time in Moscow")) \
    .select("year", "month", "day", "hour", "T")

# Обучение модели с помощью CatBoost
from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor()

catboost_model.fit(df.drop("T"), df["T"])

# Предсказание на тестовых данных
# Процесс предобработки и предсказания на тестовых данных аналогичен процессу обучения
# ...

# Закрытие сессии Spark
spark.stop()


downloading data and fitting model

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, hour
from catboost import CatBoostRegressor
import pandas as pd


import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch4",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch4",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 12
cpu = 12*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", ','.join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.21.172") # from GCP VM instances or other server ip
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default") 
         .config("spark.clickhouse.write.batchSize", "10000000")
         .config("spark.executor.memory", f"{ram}g")
         .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         .getOrCreate()
         )


clickhouse_df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""")
# Обработка данных и подготовка их для обучения модели
processed_df = clickhouse_df \
    .withColumn("year", year("Local time in Moscow")) \
    .withColumn("month", month("Local time in Moscow")) \
    .withColumn("day", dayofmonth("Local time in Moscow")) \
    .withColumn("hour", hour("Local time in Moscow")) \
    .select("year", "month", "day", "hour", "T")

# Конвертация Spark DataFrame в Pandas DataFrame
pandas_df = processed_df.toPandas()

# Разделение данных на признаки и целевую переменную
feature_columns = ["year", "month", "day", "hour"]
label_column = "T"

# Создание и обучение модели CatBoost
catboost_model = CatBoostRegressor()
catboost_model.fit(pandas_df[feature_columns], pandas_df[label_column])


Learning rate set to 0.079017
0:	learn: 10.7674753	total: 5.84ms	remaining: 5.84s
1:	learn: 10.7674753	total: 10.6ms	remaining: 5.3s
2:	learn: 10.7674753	total: 14.5ms	remaining: 4.83s
3:	learn: 10.7674753	total: 18.5ms	remaining: 4.62s
4:	learn: 10.7674753	total: 22.3ms	remaining: 4.44s
5:	learn: 10.7674753	total: 26.1ms	remaining: 4.33s
6:	learn: 10.7674753	total: 29.9ms	remaining: 4.25s
7:	learn: 10.7674753	total: 33.8ms	remaining: 4.2s
8:	learn: 10.7674753	total: 37.5ms	remaining: 4.13s
9:	learn: 10.7674753	total: 41.5ms	remaining: 4.11s
10:	learn: 10.7674753	total: 45.3ms	remaining: 4.08s
11:	learn: 10.7674753	total: 49.1ms	remaining: 4.05s
12:	learn: 10.7674753	total: 53ms	remaining: 4.02s
13:	learn: 10.7674753	total: 56.8ms	remaining: 4s
14:	learn: 10.7674753	total: 60.3ms	remaining: 3.96s
15:	learn: 10.7674753	total: 64.6ms	remaining: 3.97s
16:	learn: 10.7674753	total: 68.7ms	remaining: 3.97s
17:	learn: 10.7674753	total: 72.4ms	remaining: 3.95s
18:	learn: 10.7674753	total: 76.1

In [30]:
import joblib
  # Сохранение модели
joblib.dump(catboost_model, 'my_spark_catb_model.joblib')

['my_spark_catb_model.joblib']

In [31]:
# Загрузка модели
loaded_model = joblib.load('my_spark_catb_model.joblib')
# Загрузите новые данные и предобработайте их, извлекая признаки
# Пример: новые данные хранятся в переменной new_data
new_data = ['2024-07-30 21:00:00']



In [32]:
def temp_pred(data,model):
    # Преобразуйте новые данные в DataFrame и укажите правильное имя столбца
    data_df = pd.DataFrame(data, columns=['Local time in Moscow'])
    data_df['Local time in Moscow'] = pd.to_datetime(data_df['Local time in Moscow'], format='%Y-%m-%d %H:%M:%S')

    # Извлеките признаки из новых данных
    data_features = pd.DataFrame()
    data_features['year'] = data_df['Local time in Moscow'].dt.year
    data_features['month'] = data_df['Local time in Moscow'].dt.month
    data_features['day'] = data_df['Local time in Moscow'].dt.day
    data_features['hour'] = data_df['Local time in Moscow'].dt.hour

    # Сделайте предсказания
    predictions = model.predict(data_features)
    return predictions[0]

In [33]:
temp_pred(data=new_data, model=loaded_model)

6.457682177884218